# 科学論文 PDF 要約ツール

このノートブックでは `paper_summarizer` モジュールを使って、
科学論文の PDF を読み込み、構造化された要約を生成する方法を紹介します。

## 主な機能
- **PDF テキスト抽出**: `PyPDF2` を使用したテキスト抽出
- **セクション自動検出**: Abstract, Introduction, Methods, Results, Discussion 等を自動認識
- **TF-IDF 要約**: 各セクションから重要度の高い文を抽出
- **キーワード抽出**: 論文全体から主要なキーワードを特定
- **バッチ処理**: 複数論文の一括要約と比較

## 1. セットアップ

In [ ]:
# 必要なパッケージのインストール（初回のみ）
# !pip install PyPDF2

from paper_summarizer import (
    summarize_paper,
    analyze_paper,
    generate_report,
    extract_keywords,
    extract_text_from_pdf,
    summarize_batch,
    comparison_table,
)

print("モジュール読み込み完了")

## 2. 単一論文の要約

PDF ファイルのパスを指定して `summarize_paper()` を呼ぶだけで、
セクション別の要約レポートが生成されます。

In [ ]:
# ---- PDF ファイルのパスを指定してください ----
pdf_path = "your_paper.pdf"  # ← ここを実際のファイルパスに変更

# 要約を生成（各セクションから最大3文を抽出）
# paper = summarize_paper(pdf_path, sentences_per_section=3)

## 3. 詳細な解析

`analyze_paper()` を使うと、プログラムから各セクションの情報に
直接アクセスできます。

In [ ]:
# 詳細解析の例（PDF パスを指定して実行してください）
#
# paper = analyze_paper("your_paper.pdf")
#
# # タイトルとページ数
# print(f"タイトル: {paper.title}")
# print(f"ページ数: {paper.total_pages}")
# print(f"キーワード: {', '.join(paper.keywords)}")
# print()
#
# # 検出されたセクション一覧
# for sec in paper.sections:
#     n_sent = len(sec.sentences)
#     n_summ = len(sec.summary_sentences)
#     print(f"  [{sec.name}] 文数: {n_sent}, 要約文数: {n_summ}")

## 4. セクション別の要約文を取得

特定のセクション（例: Results）の要約文だけを取得することもできます。

In [ ]:
# 特定セクションの要約を取り出す例
#
# paper = analyze_paper("your_paper.pdf")
#
# for sec in paper.sections:
#     if sec.name in ["Abstract", "Results", "Conclusion"]:
#         print(f"\n=== {sec.name} ===")
#         for i, sent in enumerate(sec.summary_sentences, 1):
#             print(f"  {i}. {sent}")

## 5. キーワード抽出

論文全体から TF-IDF ベースで重要なキーワードを抽出します。

In [ ]:
# キーワード抽出の例
#
# raw_text, _ = extract_text_from_pdf("your_paper.pdf")
# keywords = extract_keywords(raw_text, top_n=15)
#
# print("抽出されたキーワード:")
# for i, kw in enumerate(keywords, 1):
#     print(f"  {i:2d}. {kw}")

## 6. 複数論文の一括処理と比較

ディレクトリ内のすべての PDF を一括で要約し、比較テーブルを出力できます。

In [ ]:
# バッチ処理の例
#
# papers = summarize_batch("./papers/")  # PDF が入ったディレクトリを指定
#
# # 比較テーブルの出力
# if papers:
#     table = comparison_table(papers)
#     print(table)

## 7. コマンドラインからの使用

ターミナルから直接実行することもできます:

```bash
# 単一論文
python paper_summarizer.py paper.pdf

# 複数論文
python paper_summarizer.py paper1.pdf paper2.pdf paper3.pdf
```

## デモ: モジュール動作確認

PDF ファイルがなくても、モジュールの各関数が正しく動作するか確認できます。

In [ ]:
from paper_summarizer import (
    _split_sentences,
    _detect_sections,
    _tokenize,
    _compute_tf,
    _compute_idf,
    summarize_section,
    PaperSection,
)

# サンプルテキストでテスト
sample_text = """Abstract
This study investigates the role of peptide vaccines in cancer immunotherapy.
We developed a novel neoantigen prediction pipeline using machine learning.
Our results demonstrate significant improvement in epitope binding prediction accuracy.

Introduction
Cancer immunotherapy has revolutionized oncology treatment in recent years.
Neoantigen-based vaccines represent a promising personalized approach.
However, accurate prediction of immunogenic neoantigens remains challenging.
Machine learning methods have shown potential in improving prediction accuracy.

Methods
We collected peptide-MHC binding data from the IEDB database.
A deep learning model was trained on 50000 peptide sequences.
Cross-validation was performed using a stratified 5-fold approach.
Binding affinity was measured using competitive ELISA assays.

Results
Our model achieved an AUC of 0.95 on the test dataset.
The prediction accuracy improved by 15% compared to existing methods.
We identified 23 novel neoantigen candidates from patient tumor samples.
In vitro T-cell activation assays confirmed immunogenicity for 18 candidates.

Discussion
The high prediction accuracy demonstrates the effectiveness of our approach.
Integration of structural features significantly improved binding predictions.
Our pipeline can be applied to various cancer types for personalized treatment.

Conclusion
We present a robust neoantigen prediction framework with clinical potential.
Future work will focus on clinical validation in phase I trials.

References
1. Smith et al. Nature 2023.
2. Johnson et al. Science 2024.
"""

# セクション検出
sections = _detect_sections(sample_text)
print("検出されたセクション:")
for sec in sections:
    print(f"  [{sec.name}] - 文数: {len(sec.sentences)}")

print()

In [ ]:
# IDF 計算と要約生成
all_sentences = []
for sec in sections:
    all_sentences.extend(sec.sentences)

all_docs = [_tokenize(s) for s in all_sentences]
idf = _compute_idf(all_docs)

# 各セクションの要約（References を除く）
skip = {"References", "Acknowledgements", "Supplementary"}
for sec in sections:
    if sec.name in skip:
        continue
    summary = summarize_section(sec, idf, max_sentences=2)
    print(f"\n=== {sec.name} ===")
    for i, sent in enumerate(summary, 1):
        print(f"  {i}. {sent}")